In [29]:
import json
import os
import cv2
import numpy as np
from pycocotools.coco import COCO
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.optim as optim
import torch

In [2]:
data_dir = "BrainTumorDataset"

In [3]:
def load_coco_annotations(json_file):
    """
    Loads and returns the annotations from a COCO-style JSON file.

    Args:
        json_file (str): Path to the COCO annotations JSON file.

    Returns:
        dict: Dictionary containing images, annotations, and categories.
            - images: List of image metadata.
            - annotations: List of annotation objects.
            - categories: List of category definitions.
    """
    try:
        with open(json_file, 'r') as file:
            coco_data = json.load(file)
        
        # Extract specific sections
        images = coco_data.get('images', [])
        annotations = coco_data.get('annotations', [])
        categories = coco_data.get('categories', [])
        
        return {
            'images': images,
            'annotations': annotations,
            'categories': categories
        }
    except FileNotFoundError:
        print(f"Error: File '{json_file}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON file '{json_file}'.")
        return None

In [8]:
train_annotations_path = f'{data_dir}/train/_annotations.coco.json'
valid_annotations_path = f'{data_dir}/valid/_annotations.coco.json'
test_annotations_path = f'{data_dir}/test/_annotations.coco.json'

train_annotations_dict = load_coco_annotations(train_annotations_path)

In [8]:
print(train_annotations_dict['annotations'])

[{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [145, 239, 168.75, 162.5], 'area': 27421.875, 'segmentation': [[313.75, 238.75, 145, 238.75, 145, 401.25, 313.75, 401.25, 313.75, 238.75]], 'iscrowd': 0}, {'id': 1, 'image_id': 1, 'category_id': 1, 'bbox': [194, 176, 148.75, 233.75], 'area': 34770.313, 'segmentation': [[342.5, 176.25, 193.75, 176.25, 193.75, 410, 342.5, 410, 342.5, 176.25]], 'iscrowd': 0}, {'id': 2, 'image_id': 2, 'category_id': 1, 'bbox': [133, 173, 162.5, 185], 'area': 30062.5, 'segmentation': [[295, 172.5, 132.5, 172.5, 132.5, 357.5, 295, 357.5, 295, 172.5]], 'iscrowd': 0}, {'id': 3, 'image_id': 3, 'category_id': 1, 'bbox': [245, 358, 138.75, 166.25], 'area': 23067.188, 'segmentation': [[383.75, 357.5, 245, 357.5, 245, 523.75, 383.75, 523.75, 383.75, 357.5]], 'iscrowd': 0}, {'id': 4, 'image_id': 4, 'category_id': 1, 'bbox': [80, 189, 112.5, 132.5], 'area': 14906.25, 'segmentation': [[192.5, 188.75, 80, 188.75, 80, 321.25, 192.5, 321.25, 192.5, 188.75]], 'iscrowd': 

In [59]:
def generate_segmentation_masks_brain_tumor(annotation_file, image_dir, mask_dir):
    """
    Generates binary segmentation masks for a brain tumor dataset.

    Args:
        annotation_file (str): Path to the COCO annotations JSON file.
        image_dir (str): Directory containing images.
        mask_dir (str): Directory to save generated masks.

    Returns:
        None
    """
    # Load COCO annotations
    coco = COCO(annotation_file)
    os.makedirs(mask_dir, exist_ok=True)
    
    for img_id in coco.getImgIds():
        # Load image metadata
        img_info = coco.loadImgs(img_id)[0]
        image_path = os.path.join(image_dir, img_info['file_name'])
        
        # Load annotations for the image
        ann_ids = coco.getAnnIds(imgIds=img_id)
        annotations = coco.loadAnns(ann_ids)
        
        # Create an empty binary mask
        mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
        
        for ann in annotations:
            if ann['segmentation']:
                for seg in ann['segmentation']:
                    # Convert segmentation to Nx2 array of coordinates
                    polygon = np.array(seg, dtype=np.int32).reshape((-1, 2))
                    
                    # Set mask based on category_id
                    if ann['category_id'] == 1:  # No tumor
                        cv2.fillPoly(mask, [polygon], color=0)  # Explicitly set to background (no tumor)
                    elif ann['category_id'] == 2:  # Tumor present
                        cv2.fillPoly(mask, [polygon], color=1)  # Explicitly set to foreground (tumor)

        # Validate mask values (should only contain 0 and 1)
        mask[mask > 1] = 1
        
        # Save the mask
        mask_filename = f"{img_info['file_name'].split('.jpg')[0]}_mask.png"
        mask_path = os.path.join(mask_dir, mask_filename)
        cv2.imwrite(mask_path, mask)  # Save as grayscale (0 and 1)

In [60]:
# Create masks for the train, validation, and test datasets
train_image_path = f'{data_dir}/train/images/'
train_mask_path = f'{data_dir}/train/mask/'
valid_image_path = f'{data_dir}/valid/images/'
valid_mask_path = f'{data_dir}/valid/mask/'
test_image_path = f'{data_dir}/test/images/'
test_mask_path = f'{data_dir}/test/mask/'

# generate_segmentation_masks_brain_tumor(train_annotations_path, train_image_path, train_mask_path)
generate_segmentation_masks_brain_tumor(valid_annotations_path, valid_image_path, valid_mask_path)
generate_segmentation_masks_brain_tumor(test_annotations_path, test_image_path, test_mask_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [56]:
class BrainTumorDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, target_size=(256, 256)):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_filenames = os.listdir(image_dir)
        self.mask_filenames = os.listdir(mask_dir)
        self.transform = transform
        self.target_size = target_size

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # Load image and mask
        image_path = os.path.join(self.image_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_filenames[idx])

        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path)

        # Resize both image and mask to the target size
        resize = transforms.Resize(self.target_size)
        image = resize(image)
        mask = resize(mask)

        # Convert mask to tensor and ensure it's in long format for loss
        mask = torch.tensor(np.array(mask), dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, mask

In [44]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [61]:
# Create the datasets and dataloaders
train_dataset = BrainTumorDataset(train_image_path, train_mask_path, train_transform)
valid_dataset = BrainTumorDataset(valid_image_path, valid_mask_path, valid_transform)
test_dataset = BrainTumorDataset(test_image_path, test_mask_path, test_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [26]:
# Load the pretrained resnet model and adapt for segmentation
model = fcn_resnet50(weights=FCN_ResNet50_Weights.DEFAULT)
model.classifier[4] = nn.Conv2d(512, 2, kernel_size=1)

In [37]:
# Set hyperparameters
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
decay = 0.0001
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=decay)
num_epochs = 1

In [58]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Store the model with the best performance on the validation set
best_val_loss = float('inf')
best_model_path = "best_model.pth"  # Path to save the best model based on validation loss

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss:.4f}")

    # Evaluate performance on validation dataset
    model.eval()
    valid_loss = 0

    with torch.no_grad():
        for images, masks in valid_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            valid_loss += loss.item()

    avg_valid_loss /= len(valid_loader)
    print(f"Validation Loss: {valid_loss:.4f}")

    # Save the model if it has the lowest validation loss
    if avg_valid_loss < best_val_loss:
        best_val_loss = avg_valid_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with validation loss: {best_val_loss:.4f}")

Epoch [1/1], Loss: 0.0507


IndexError: Target 37 is out of bounds.